In [6]:
import networkx as nx
import random
from collections import Counter
import numpy as np

"""
Authors:
 - Nikolaj Thorsen, nivt@itu.dk
 - Lukas Ditlevsen, lukd@itu.dk
 - Jonas Hansen, jonha@itu.dk
"""

'\nAuthors:\n - Nikolaj Thorsen, nivt@itu.dk\n - Lukas Ditlevsen, lukd@itu.dk\n - Jonas Hansen, jonha@itu.dk\n'

In [7]:
# RANDOM SURFER
def file_open(filename):
    fh=open(filename, 'rb')
    G=nx.read_adjlist(fh, create_using=nx.DiGraph())
    fh.close()
    return G
    
FILENAMES = ['bigRandom.txt','medium.txt','p2p-Gnutella08-mod.txt','three.txt','tiny.txt','wikipedia.txt'] 
# NEW: variable that stores the filenames (remember to merge)


G=file_open(FILENAMES[2]) 


def surfer(m,n):
    nodeV = [0 for i in range(len(list(G)))]
    
    # Define starting point for the random surfer simulation
    x_current = random.choices(sorted([int(nodes) for nodes in G.nodes()]))
    
    #initiate for loop that runs n iterations
    for e in range(n):

        #initiate if statement that changes it's current position to a random node in the net, should a random value
        # fall equal to or below the given damping factor, or the current node is a dangeling node 
        if random.random() <= m or bool(list(G.neighbors(str(x_current[0])))) == False:

            # list that holds nodes sorted by their number
            g = sorted([int(nodes) for nodes in G.nodes()])

            # remove the current node, so as to make sure that, when changing nodes, the new starting point is not the same
            # node from which the change was initiated
            g.pop(x_current[0])

            # change to a random node
            x_current = random.choices(g)
            nodeV[x_current[0]:x_current[0]+1] = [nodeV[x_current[0]]+1]
        
        # initiate else-statement that, in the case the simulation did not change to a random node, kept the simulation going
        else:
            x_current = [int(random.choices(list(G.neighbors(str(x_current[0]))))[0])]
            nodeV[x_current[0]:x_current[0]+1] = [nodeV[x_current[0]]+1]
    
    # return a sorted list of the nodes most visited during the simulation
    return sorted(range(len(nodeV)), key=lambda i: nodeV[i])[-10:][::-1]

x = surfer(0.15,100000)
print(x)

# in order from least to most popular, how to reverse?

In [3]:
def PageRank(G,m,n,t=0.00000001):
        # Creating the A matrix by converting the netgraph of G to a numpy array
        A = nx.to_numpy_array(G)

        # Locates and changes all rows representing dangling nodes to boolean value
        degree=np.sum(A,axis=1,keepdims=True)
        A=A+(degree==0)

        # Normalizes nodes with outgoing edges, so that each entry becomes one divided by the nummber outgoing edges
        degree=np.sum(A,axis=1, keepdims=True)
        A=A/degree

        # Transposing A to make A coloumn-stochastic instead of row-stochastic
        A = A.T

        # Making the S matrix the dimensions of which is n by n, where n is the number of nodes in the graph
        S = np.zeros([len(G),len(G)])

        # We fill the S matrix with 1 divided by n in all entries
        S.fill(1/len(G))

        # Making M matrix
        M = (1-m)*A+m*S

        # Making X matrix, i.e. x0 - the initial pagerank for all pages, where each page is of equal importance
        X = np.zeros([len(G),1])
        X.fill(1/len(G))

        # Calculating M@X n times, where we expect to get a better and better result for each iteration, until it eventually converges
        for i in range(n):
            x_1 = M@X
            d = np.abs(X-x_1)
            if np.mean(d) < t:
                return i, X
            X = x_1
        return i, X



# we store the output of the pagerank function in a variable x, which outputs to be a column array of dimensions len(G) by 1
iterations, x = PageRank(G,0.15,100)

#
d = dict(zip(G.nodes(), x.flatten()))

# Sort d in terms of the values of the keys - return dict with the ten most popular pages
dic=dict((Counter(d)).most_common(10))
dic, iterations

({'367': 0.0023876508300157556,
  '249': 0.0021842951424048823,
  '145': 0.00205488593932474,
  '264': 0.0019987499161941607,
  '266': 0.0019634403057468244,
  '123': 0.0018634931279900295,
  '127': 0.001860511078967605,
  '122': 0.001853246070158606,
  '1317': 0.0018435046570815905,
  '5': 0.0018310738884673387},
 8)

In [5]:
l = nx.pagerank(G,alpha=0.85)
dict((Counter(l)).most_common(10))

AttributeError: module 'scipy.sparse' has no attribute 'coo_array'